In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# Clear any logs from previous runs
rm -rf ./logs/

SyntaxError: invalid syntax (<ipython-input-2-0cddaa2dbc5f>, line 2)

In [3]:
import tensorflow as tf  ## import tensorflow
import datetime
from tensorflow.keras.models import Sequential  ## import the neural network model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D  ## import keras layers

In [4]:
import contrib
from tensorflow import lite

In [5]:
from tensorflow import keras

In [6]:
import pickle  ## import pickle to load data
x_train = pickle.load(open("x_train.pickle","rb"))
y_train = pickle.load(open("y_train.pickle","rb"))
x_val = pickle.load(open("x_val.pickle","rb"))
y_val = pickle.load(open("y_val.pickle","rb"))

In [7]:
import numpy as np    ## normalaze the data

x_train = np.array(x_train)
y_train = np.array(y_train) 

x_val = np.array(x_val)
y_val = np.array(y_val) 

In [63]:
print(x_val.shape[1:])

(24, 24)


In [19]:
model = tf.keras.models.Sequential()

model.add(Flatten())
model.add(Dense(500,activation = tf.nn.relu))
model.add(Dense(12,activation = tf.nn.softmax))

model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(x_train,
          y_train,
          epochs = 40,
          batch_size = 1000,
          validation_data = (x_val, y_val),
          callbacks=[tensorboard_callback])

%tensorboard --logdir logs/fit

Epoch 1/40
6/6 [==============================] - 0s 40ms/step - loss: 2.1606 - accuracy: 0.3061 - val_loss: 1.4657 - val_accuracy: 0.5900
Epoch 2/40
6/6 [==============================] - 0s 15ms/step - loss: 1.1311 - accuracy: 0.6861 - val_loss: 0.9318 - val_accuracy: 0.7582
Epoch 3/40
6/6 [==============================] - 0s 15ms/step - loss: 0.7308 - accuracy: 0.7972 - val_loss: 0.7111 - val_accuracy: 0.8291
Epoch 4/40
6/6 [==============================] - 0s 15ms/step - loss: 0.5571 - accuracy: 0.8652 - val_loss: 0.5721 - val_accuracy: 0.8536
Epoch 5/40
6/6 [==============================] - 0s 15ms/step - loss: 0.4287 - accuracy: 0.8925 - val_loss: 0.4916 - val_accuracy: 0.8745
Epoch 6/40
6/6 [==============================] - 0s 16ms/step - loss: 0.3528 - accuracy: 0.9165 - val_loss: 0.4455 - val_accuracy: 0.8936
Epoch 7/40
6/6 [==============================] - 0s 15ms/step - loss: 0.2794 - accuracy: 0.9440 - val_loss: 0.3856 - val_accuracy: 0.9173
Epoch 8/40
6/6 [===========

Reusing TensorBoard on port 6006 (pid 3121), started 0:08:44 ago. (Use '!kill 3121' to kill it.)

In [23]:
### TensorFlow Lite
model.save('saved')

saved_model_dir = 'line.h5'
tf.keras.models.save_model(model,saved_model_dir)

saved_model_dir = 'saved'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

model = tf.keras.models.load_model('line.h5')